In [ ]:
#Commands for installing graph-tool package on Google Colab
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [ ]:
#python3-cairo from Ubuntu's reposity is linked with a different python version; we need to improvise
#goal: recreate panther.py using graph-tool and colab
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

In [ ]:
from graph_tool.all import *
from graph_tool.topology import *

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:1500: RuntimeWarning: Error importing Gtk module: cannot import name '_gi' from partially initialized module 'gi' (most likely due to a circular import) (/usr/lib/python3/dist-packages/gi/__init__.py); GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


13936676

# New Section

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns= ['source', 'target', 'weight'])

for chunk in pd.read_csv('input.csv', chunksize=100000):
  df = pd.concat([df, chunk])
  df = df.groupby('source').head(100)
  df = df.groupby('target').head(100)

In [ ]:
import time
import pandas as pd
import csv
import numpy as np
import math


def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

start_time = time.time()

sim_dict = {}

start = time.time()

#print(G.edges(node, data="weight"))
graph_vertices = {}

#store indices of all graph vertices

for v in g.iter_vertices():  
  graph_vertices[v] = None



while len(graph_vertices) > 0:

    start_node = int(next(iter(graph_vertices)))

    panther_list = {start_node}

    i = 0

    
    neighbors = [v for v in g.iter_all_neighbors(start_node)]

    print(len(neighbors))

    for neighbor in neighbors:
      if neighbor not in panther_list:
        panther_list.add(neighbor)
    
    
    #get second-degree neighbors of node
    
    for neighbor in neighbors:
        second_neighbors = [int(v) for v in g.iter_all_neighbors(neighbor)]

        if len(panther_list) > 2500:
            break
        for n in second_neighbors:
            if n not in panther_list:
                panther_list.add(n)
        
        for second in second_neighbors: 
          third_neighbors = [int(v) for v in g.iter_all_neighbors(second)]

          if len(panther_list) > 2500:
            break
          for n in third_neighbors:
            if n not in panther_list:
              panther_list.add(n)
    
    array_list = [] 

    proto_dict = {}

    new_node = [1 if v in graph_vertices else 0 for v in panther_list]

    node_count = sum(new_node)

    panther_list = [x for _, x in sorted(zip(new_node, panther_list))]

    panther_list.reverse()

    

    def filter_node(n):
      return int(n) in panther_list
    
    
    panther_subgraph = GraphView(g, vfilt=filter_node)

    subgraph_vertices = [v for v in panther_subgraph.iter_vertices()]

    for v in panther_list:
        proto_dict[v] = 0

    for v in panther_list:
        node_dict = proto_dict.copy()
        node_vec = []
        
        for s, t, i in panther_subgraph.iter_all_edges(v, [weight]):
            
            if int(s) == int(v):
              connect = t
            elif int(t) == int(v):
              connect = s
            else: 
              connect = s

            node_dict[connect] = i


        node_vec = list(node_dict.values())

        array_list.append(np.array(node_vec))


#iterate over array list, for each vector find the most similar other vectors
#memoize responses
    
    

    for i in range(node_count):

        
        similarities = []
        
        for j in range(len(array_list)):
            sim = cos_sim(array_list[i], array_list[j])
            similarities.append( (vmap[panther_list[j]], sim) )

        similarities = sorted(similarities, key = lambda x: 100 if math.isnan(x[1]) else - x[1]) #put the similarities list in desc order
        if len(similarities) > 51:
            similarities = similarities[1:51] # find top 50 similarities, excluding itself
        else:
            similarities = similarities[1:]

        vertex = vmap[panther_list[i]]  
            
        sim_dict[vertex] = list(similarities)
        print(len(sim_dict))
    
    remove_list = panther_list[:node_count]

    for node in remove_list:
      graph_vertices.pop(node)

end = time.time()
print("total time: " + str(end - start))

    #panther_graph.add_nodes_from(panther_list)

    #iterate over subgraph nodes
    # create list of connections
    #consider Jaccard coefficient and cosine similarity
    #use graph.get_edge_data
    
    # changes: don't remove vertices from graph; don't calculate similarities unless
    # you haven't yet. Calculate 100 new similarity scores at a time. Keep
    # graph copy where you're filtering out old nodes.




                            


In [ ]:
#get the top 1000 vertices by pagerank
pt = pagerank(g)

pair_list = [(v, pt[v]) for v in g.iter_vertices()]
pair_list = sorted(pair_list, key=lambda x: x[1], reverse=True)

top_vertices = [p[0] for p in pair_list[:1000]]

pr_subgraph = GraphView(g, vfilt=lambda x: int(x) in top_vertices)

In [ ]:
# generate force-directed graph

pos = fruchterman_reingold_layout(pr_subgraph, n_iter=100)
graph_draw(pr_subgraph, pos=pos, output="graph-draw-fr.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fbf0474ad00, at 0x7fbf0474a5e0>

87470


In [ ]:
import csv
coords = []
for v in pr_subgraph.iter_vertices():
  dot_dict = {"Key": v, "X": pos[v][0], "Y": pos[v][1]}
  coords.append(dot_dict)

field_names = ["Key", "X", "Y"]

with open('node_coords.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(coords)

IndexError: ignored

In [ ]:
import csv
with open('similarities.csv', 'w', newline='') as csvfile:
  header = ['track']
  header = header + list(range(1, 51))
  writer = csv.DictWriter(csvfile, fieldnames=header)
  for key in sim_dict.keys():
    row = {}
    row['track'] = key
    for i in range(50):
      if len(sim_dict[key]) > i:
        row[i + 1] = sim_dict[key][i]
    
    writer.writerow(row)

[('1bWFvGIN7b5cTps09X6tZ8', 0.4590780850487671), ('2JyxMvRn57BdTCCSpGlKLG', 0.37707334353833644), ('65D95SmygxGCQGiI64eaZf', 0.365554032416927), ('5TpaWJKnuyA4MjzAbFXSTQ', 0.3177762847994256), ('3XabgBQYC7H80agKcAq83Y', 0.30834039177742634), ('09XsCShQ3oCA3a5QT5ZUG5', 0.3074558940517994), ('24PWKmemCvqfyVXODhoKHW', 0.30014336268406405), ('0EgigrGFGb4PHaVNb7fgK7', 0.2965990580714912), ('6D1CstH1YvzNN8cPd9ay9Q', 0.2815624608409983), ('6YTp4sUEi3oWNToQvOjke6', 0.2650725101465462), ('6JJ8eDRnrwORwTi6K2KFOx', 0.2631099209781971), ('5tEbnKMOMK8LLoOIsEkWRr', 0.2613801048830039), ('1LMAQunVS6Dq8wEqQnsL8Y', 0.2506147601684105), ('76dZuMVXjIyLPRLv00skGd', 0.24529926644357516), ('1NpcguIbdLR25tlymnwVVC', 0.24032578213897166), ('2boJnT3S2aSBagFEUTXrfx', 0.23374737492215086), ('0BjsCH9fQk9Y2c4uFhMFdV', 0.22568514811147514), ('6QwwsJC57SitPcWuyEpuwd', 0.22510994381306007), ('6KoK723c5pmYKiHvH69JDr', 0.22229846934137182), ('0LDsM5oTo0gKnrRMS9z03e', 0.215471208861415), ('0DW5anNzTO7h0OlKqFsVQ6', 0.214

In [ ]:
# for each node, find its neighborhood. find similarities for all the nodes that haven't been 
# counted yet.